In [1]:
import os
import pickle
import pandas as pd

In [2]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
df = pd.read_csv("EuroparlSentToPos.csv", encoding="UTF-8", sep=";")
df_filtered = df.dropna(subset=["pos"])
sentences = list(df_filtered["sent"])
all_pos = list(df_filtered["pos"])
df_filtered

,sent,pos
0,Genoptagelse af sessionen,NOUN ADP NOUN
1,"Jeg erklærer Europa-Parlamentets session, der ...",PRON VERB NOUN PUNCT PRON AUX VERB NOUN DET AD...
2,"Som De kan se, indfandt det store ""år 2000-pro...",ADP PRON AUX VERB VERB DET ADJ NOUN X PRON PUN...
3,De har udtrykt ønske om en debat om dette emne...,PRON AUX VERB NOUN ADP DET NOUN ADP DET NOUN A...
4,I mellemtiden ønsker jeg - som også en del kol...,ADP NOUN VERB PRON PUNCT ADP ADV DET NOUN NOUN...
...,...,...
1549995,"Hr. formand, kære kolleger, De husker måske, a...",PROPN X ADJ PUNCT PRON VERB PUNCT SCONJ PRON A...
1549996,Nu blev jeg for to uger siden ringet op af Par...,ADV AUX PRON ADP NUM NOUN ADV VERB ADV ADP NOU...
1549997,Jeg kom således forventningsfuld til Strasbour...,PRON VERB ADV ADJ ADP PROPN VERB ADP DET NOUN ...
1549998,På kanal 12 havde vi nu teleshopping med tyske...,ADP NOUN NUM VERB PRON ADV NOUN ADP ADJ NOUN P...


In [3]:
def find_det_noun_pairs(lst):
    indexes = []
    for i in range(len(lst)):
        if lst[i] == "DET":
            for j in range(i+1, len(lst)):
                if lst[j] == "NOUN":
                    if all(x in ["ADV", "ADJ"] for x in lst[i+1:j]):
                        indexes.append((i, j))
                    break
    return indexes

In [4]:
testset = []
fælleskøn_labels = []
testset_sentences = []

for sent, pos in zip(sentences, all_pos):
    indexes = find_det_noun_pairs(pos.split())
    words = sent.split()
    for index in indexes:
        if words[index[0]][-1] == "n":
            fælleskøn_labels.append(True)
            testset.append(words[index[0]:index[1]+1])
            testset_sentences.append(sent)
        elif words[index[0]][-1] == "t":
            fælleskøn_labels.append(False)
            testset.append(words[index[0]:index[1]+1])
            testset_sentences.append(sent)


In [5]:
len(testset)

1301401

In [6]:
with open("GenderDict.pickle", "rb") as f:
    genderDict = pickle.load(f)

In [8]:
def get_stats(output=False):
    correct = 0
    total = 0
    no_guess = 0
    wrongs = []

    for set, label in zip(testset, fælleskøn_labels):
        if set[-1] in genderDict and set[0] in ["en", "et", "den", "det"]:
            total += 1
            if label == genderDict[set[-1]]:
                correct += 1
            else:
                wrongs.append((set, set[-1], genderDict[set[-1]]))
                print(set, set[-1], genderDict[set[-1]])
        else:
            no_guess += 1
    if output: 
        print(f"Accuracy:  {round(correct/total*100, 2)} %")
        print(f"No guess: ", round(no_guess/len(testset)*100, 2), "%")
        print(f"Number of wrong: {len(wrongs)}")
    return len(wrongs), wrongs

nwrongs, wrongs = get_stats(output=True)

['en', 'europæisk', 'materiel'] materiel False
['et', 'katalog'] katalog True
['et', 'slags'] slags True
['det', 'nuværende', 'koncept'] koncept True
['et', 'eventuelt', 'misbrug'] misbrug True
['et', 'slags'] slags True
['det', 'gud'] gud True
['et', 'slags'] slags True
['et', 'kritik'] kritik True
['et', 'katalog'] katalog True
['et', 'lige', 'så', 'stort', 'katalog'] katalog True
['det', 'offentliges', 'interesse'] interesse True
['et', 'større', 'koordinering'] koordinering True
['det', 'fejlagtige', 'liberalistiske', 'koncept'] koncept True
['det', 'godes', 'tjeneste'] tjeneste True
['det', 'første', 'indførelse'] indførelse True
['det', 'tredje', 'indførelse'] indførelse True
['et', 'række'] række True
['en', 'skridt'] skridt False
['det', 'offentliges', 'side'] side True
['et', 'virus'] virus True
['et', 'ordet', 'etik'] etik True
['et', 'vagt'] vagt True
['et', 'katalog'] katalog True
['det', 'tekniske', 'design'] design True
['et', 'fast', 'koncept'] koncept True
['det', 'nati

In [11]:
def save_dict(genderDict):
    with open("GenderDictChecked.pickle", "wb") as f:
        pickle.dump(genderDict, f)

In [15]:
prev_nwrongs = nwrongs
while prev_nwrongs > 0:
    print(wrongs[0])
    print(f"Prev nwrongs: {prev_nwrongs}")
    should_correct = input("1 for change, 2 for delete, 3 for skip, 4 for break:\n ")
    if should_correct == "1":
        genderDict[wrongs[0][0][-1]] = not genderDict[wrongs[0][0][-1]]
    elif should_correct == "2":
        genderDict.pop(wrongs[0][0][-1])
    elif should_correct == "4":
        break
    nwrongs, wrongs = get_stats()
    if nwrongs > prev_nwrongs:
        print("Rejected!")
    print(f"Prev nwrongs: {prev_nwrongs}, now nwrongs: {nwrongs}")
    prev_nwrongs = nwrongs
    save_dict(genderDict)

(['et', 'unødvendigt', 'brud'], 'brud', True)
Prev nwrongs: 288
['et', 'unødvendigt', 'brud'] brud True
['en', 'europæisk', 'materiel'] materiel False
['et', 'katalog'] katalog True
['det', 'femte', 'brug'] brug True
['det', 'gode', 'plan'] plan True
['et', 'brud'] brud True
['et', 'slags'] slags True
['det', 'nuværende', 'koncept'] koncept True
['et', 'led'] led True
['et', 'mere', 'overordnet', 'plan'] plan True
['et', 'mere', 'menneskeligt', 'plan'] plan True
['det', 'væv'] væv True
['et', 'trekløver'] trekløver True
['det', 'døve', 'øre'] øre True
['et', 'lavere', 'plan'] plan True
['et', 'eventuelt', 'misbrug'] misbrug True
['et', 'brud'] brud True
['et', 'slags'] slags True
['det', 'gud'] gud True
['et', 'slags'] slags True
['det', 'felt'] felt True
['et', 'kritik'] kritik True
['et', 'katalog'] katalog True
['et', 'lige', 'så', 'stort', 'katalog'] katalog True
['det', 'første', 'brug'] brug True
['det', 'sigte'] sigte True
['det', 'finansielle', 'plan'] plan True
['det', 'mindst